## The notebook implementation of Research paper summarizer:

---

In [ ]:
!pip install transformers datasets torch

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name,clean_up_tokenization_spaces= True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("scientific_papers", "arxiv")

README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6440
    })
})


In [4]:
def preprocess_func(text):
    inputs = text['article']
    targets= text['abstract']
    model_inputs = tokenizer(inputs, max_length = 1024, truncation= True, padding= 'max_length')
    labels = tokenizer(targets, max_length= 256, truncation= True, padding= 'max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset= dataset.map(preprocess_func, batched= True)

Map:   0%|          | 0/203037 [00:00<?, ? examples/s]

Map:   0%|          | 0/6436 [00:00<?, ? examples/s]

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [13]:
# #@ Saving the mapped dataset so that it becomes effecient if training error occours 

# tokenized_dataset.save_to_disk("mapped_dataset_dir")

Saving the dataset (0/18 shards):   0%|          | 0/203037 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6436 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
# !zip -r compressed_mapped.zip mapped_dataset_dir

Okay, now I'm not mapping it again. Instead, I'll unzip the mapped_value and run from this. 

In [5]:
from datasets import load_from_disk

mapped_df= load_from_disk('/kaggle/input/compressed-mapped-files/mapped_dataset_dir')

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [6]:
from transformers import Trainer, TrainingArguments

training_args= TrainingArguments(
    output_dir= "./results",
    eval_strategy= "epoch", 
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate= 2e-5, 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs= 3, 
    weight_decay = 0.01, 
    logging_dir="./logs",
    fp16 = True
)

In [8]:
# small_train = tokenized_dataset["train"].select(range(10000))
# small_eval = tokenized_dataset["validation"].select(range(4000))

In [7]:
small_train = mapped_df["train"].select(range(10000))
small_eval = mapped_df["validation"].select(range(2000))

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [10]:
import wandb
wandb.login(key= secret_value_0)
wandb.init(project="Researchpaper_summarize", name="wtf")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: firojpaudel (firojpaudel-madan-bhandari-memorial-college). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,2.141400,1.818125
2,1.867500,1.777878
3,1.785800,1.769484


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-mod

TrainOutput(global_step=1875, training_loss=1.8924777669270834, metrics={'train_runtime': 10015.5745, 'train_samples_per_second': 2.995, 'train_steps_per_second': 0.187, 'total_flos': 6.501313806336e+16, 'train_loss': 1.8924777669270834, 'epoch': 3.0})

In [28]:
import shutil
import os
logs_dir = '/kaggle/working/results'
try:
    if os.path.exists(logs_dir):
        shutil.rmtree(logs_dir)  # Remove the directory and its contents
        print(f"Directory '{logs_dir}' removed successfully.")
    else:
        print(f"Directory '{logs_dir}' does not exist.")
except OSError as e:
    print(f"Error removing directory '{logs_dir}': {e}")

Directory '/kaggle/working/results' removed successfully.


In [14]:
##@ saving the finetuned model:
trainer.save_model("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine_tuned_bart/tokenizer_config.json',
 './fine_tuned_bart/special_tokens_map.json',
 './fine_tuned_bart/vocab.json',
 './fine_tuned_bart/merges.txt',
 './fine_tuned_bart/added_tokens.json',
 './fine_tuned_bart/tokenizer.json')

In [17]:
import os

print(os.listdir("./fine_tuned_bart"))

['model.safetensors', 'training_args.bin', 'special_tokens_map.json', 'vocab.json', 'tokenizer_config.json', 'tokenizer.json', 'config.json', 'generation_config.json', 'merges.txt']


all the weights are there in model.safetensors and training arguments in training_args.bins

In [19]:
#@ Testing once before downloading the model 

model= AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/fine_tuned_bart')
tokenizer= AutoTokenizer.from_pretrained('/kaggle/working/fine_tuned_bart')

In [20]:
def summarize(testing):
    #@ Tokenizng the inp text to test in the finetuned model
    inputs = tokenizer(testing, max_length= 1024, truncation= True, return_tensors='pt')

    #@ Generating the summary
    summary_enc = model.generate(inputs['input_ids'], max_length= 500, min_length= 250, length_penalty= 2.0, num_beams=4, early_stopping= True)

    #@ Decoding the generated summary
    summary_fin = tokenizer.decode(summary_enc[0], skip_special_tokens= True)

    return summary_fin

In [24]:
input = '''
 1 Introduction
 Language model pre-training has been shown to
 be effective for improving many natural language
 processing tasks (Dai and Le, 2015; Peters et al.,
 2018a; Radford et al., 2018; Howard and Ruder,
 2018). These include sentence-level tasks such as
 natural language inference (Bowman et al., 2015;
 Williams et al., 2018) and paraphrasing (Dolan
 and Brockett, 2005), which aim to predict the re
lationships between sentences by analyzing them
 holistically, as well as token-level tasks such as
 named entity recognition and question answering,
 where models are required to produce fine-grained
 output at the token level (Tjong Kim Sang and
 De Meulder, 2003; Rajpurkar et al., 2016).
 There are two existing strategies for apply
ing pre-trained language representations to down
stream tasks: feature-based and fine-tuning. The
 feature-based approach, such as ELMo (Peters
 et al., 2018a), uses task-specific architectures that
 include the pre-trained representations as addi
tional features. The fine-tuning approach, such as
 the Generative Pre-trained Transformer (OpenAI
 GPT) (Radford et al., 2018), introduces minimal
 task-specific parameters, and is trained on the
 downstream tasks by simply fine-tuning all pre
trained parameters. The two approaches share the
 sameobjective function during pre-training, where
 they use unidirectional language models to learn
 general language representations.
 We argue that current techniques restrict the
 power of the pre-trained representations, espe
cially for the fine-tuning approaches. The ma
jor limitation is that standard language models are
 unidirectional, and this limits the choice of archi
tectures that can be used during pre-training. For
 example, in OpenAIGPT,theauthors use aleft-to
right architecture, where every token can only at
tend to previous tokens in the self-attention layers
 of the Transformer (Vaswani et al., 2017). Such re
strictions are sub-optimal for sentence-level tasks,
 and could be very harmful when applying fine
tuning based approaches to token-level tasks such
 as question answering, where it is crucial to incor
porate context from both directions.
 In this paper, we improve the fine-tuning based
 approaches by proposing BERT: Bidirectional
 Encoder Representations from Transformers.
 BERT alleviates the previously mentioned unidi
rectionality constraint by using a “masked lan
guage model” (MLM) pre-training objective, in
spired by the Cloze task (Taylor, 1953). The
 masked language model randomly masks some of
 the tokens from the input, and the objective is to
 predict the original vocabulary id of the masked

'''

In [25]:
summary = summarize(input) 

In [26]:
summary

' language model pre-training has been shown to be effective for improving many natural language processing tasks (Dai and Le, 2015; Peters et al., \n 2018a; Radford et al. \n, 2018b; Howard and Ruder, 2018c ). \n these include sentence-level tasks such as natural language inference and paraphrasing, where models are required to produce fine-grained output at the token level (Tjong Kim Sang and \n De Meulder, 2003; Rajpurkar et al,. 2016). \n there are two existing strategies for applying pre-trained language representations to downstream tasks: feature-based and fine-tuning. The two approaches share the sameobjective function during pre- training, where they use unidirectional language models to learn general language representations (e.g. the Generative Pre-trained Transformer (OpenAI \n GPT) and ELMo (Peters et al, 2018a) ). in this paper \n we argue that current techniques restrict the power of the pre- trained representations by limiting the choice of archi \n tectures that can be

Humm Its working....

In [29]:
##@ Zipping the fine-tuned folder in order to download...
!zip -r compressed_finttuned.zip /kaggle/working/fine_tuned_bart

  adding: kaggle/working/fine_tuned_bart/ (stored 0%)
  adding: kaggle/working/fine_tuned_bart/model.safetensors (deflated 7%)
  adding: kaggle/working/fine_tuned_bart/training_args.bin (deflated 51%)
  adding: kaggle/working/fine_tuned_bart/special_tokens_map.json (deflated 52%)
  adding: kaggle/working/fine_tuned_bart/vocab.json (deflated 59%)
  adding: kaggle/working/fine_tuned_bart/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/fine_tuned_bart/tokenizer.json (deflated 72%)
  adding: kaggle/working/fine_tuned_bart/config.json (deflated 61%)
  adding: kaggle/working/fine_tuned_bart/generation_config.json (deflated 48%)
  adding: kaggle/working/fine_tuned_bart/merges.txt (deflated 53%)
